In [70]:
from salesforce_bulk import SalesforceBulk, CsvDictsAdapter
from salesforce_bulk.util import IteratorBytesIO
from simple_salesforce import Salesforce
import pandas as pd
import os
import json
from time import sleep

bulk = SalesforceBulk(
    username=os.getenv("SF_USERNAME"),
    password=os.getenv("SF_PASSWORD"),
    security_token=os.getenv("SF_TOKEN")
)

ss_client = Salesforce(
    username=os.getenv("SF_USERNAME"),
    password=os.getenv("SF_PASSWORD"),
    security_token=os.getenv("SF_TOKEN")
)

In [66]:
job = bulk.create_query_job("Property__c", contentType='JSON')
batch = bulk.query(job, "select Id from Property__c")
bulk.close_job(job)
while not bulk.is_batch_done(batch):
    sleep(10)

records = []
for result in bulk.get_all_results_for_query_batch(batch):
    result = json.load(IteratorBytesIO(result))
    for row in result:
        records.append(row) # dictionary rows

In [79]:
len(records)

10007

In [83]:
csv_iter = CsvDictsAdapter(iter([{'Id':x['Id'] for x in records[:5000]}]))

In [87]:
[{'Id':x['Id'] for x in records[:10]}]

[{'Id': 'a00Hp000018kStUIAU'}]

In [76]:
# # Delete all records using the IDs retrieved.
delete_job = bulk.create_delete_job("Property__c", contentType='CSV')

batch = bulk.post_batch(delete_job, csv_iter)


In [77]:
bulk.wait_for_batch(delete_job, batch)
bulk.close_job(delete_job)

In [15]:
result

<generator object SalesforceBulk.get_all_results_for_query_batch at 0xffff5f6317e0>

In [78]:
ss_client.query("Select count(Name) from Property__c")

OrderedDict([('totalSize', 1),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'AggregateResult')])),
                            ('expr0', 10006)])])])